Realizada a EDA e tratamento dos dados, realizarei o treinamento dos modelos. Testarei dois modelos: Regressão Logística e XGBoost.

In [1]:
import pandas as pd 
import numpy as np 

## Leitura dos dados

In [2]:
dados=pd.read_csv('dados_tratados/basecompleta.csv')

In [3]:
dados.columns

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Cabine_nula', 'Treino',
       'Sex_male', 'Embarked_Q', 'Embarked_S', 'Titulo_Col.',
       'Titulo_Countess.', 'Titulo_Don.', 'Titulo_Dona.', 'Titulo_Dr.',
       'Titulo_Jonkheer.', 'Titulo_Lady.', 'Titulo_Major.', 'Titulo_Master.',
       'Titulo_Miss.', 'Titulo_Mlle.', 'Titulo_Mme.', 'Titulo_Mr.',
       'Titulo_Mrs.', 'Titulo_Ms.', 'Titulo_Rev.', 'Titulo_Sir.',
       'letra_cabine_B', 'letra_cabine_C', 'letra_cabine_D', 'letra_cabine_E',
       'letra_cabine_F', 'letra_cabine_G', 'letra_cabine_Nula',
       'letra_cabine_T'],
      dtype='object')

## Normalização das variáveis

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
scaler = StandardScaler()
dados['Age_scaled'] = scaler.fit_transform(dados['Age'].values.reshape(-1,1))
dados['Fare_scaled'] = scaler.fit_transform(dados['Fare'].values.reshape(-1, 1))

In [6]:
# Separando base de treino e base de teste
base_treino = dados[dados['Treino'] == 1]
base_teste = dados[dados['Treino'] ==0]

In [7]:
base_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 891 to 1308
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         418 non-null    int64  
 1   PassengerId        418 non-null    int64  
 2   Survived           0 non-null      float64
 3   Pclass             418 non-null    int64  
 4   Name               418 non-null    object 
 5   Age                418 non-null    float64
 6   SibSp              418 non-null    int64  
 7   Parch              418 non-null    int64  
 8   Ticket             418 non-null    object 
 9   Fare               418 non-null    float64
 10  Cabin              91 non-null     object 
 11  Cabine_nula        418 non-null    int64  
 12  Treino             418 non-null    int64  
 13  Sex_male           418 non-null    int64  
 14  Embarked_Q         418 non-null    int64  
 15  Embarked_S         418 non-null    int64  
 16  Titulo_Col.        418 

Irei tirar as colunas que não irei usar no treinamento do modelo.

In [8]:
base_treino.drop(['Unnamed: 0', 'Cabin', 'Name', 'Ticket'], axis=1, inplace=True)
base_teste.drop(['Unnamed: 0', 'Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

C:\Users\art19\anaconda3\envs\TitanicProject\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Separando os features da variável target.

In [11]:
X = base_treino.drop(columns=['Survived', 'Age', 'Fare'], axis=1)
y = base_treino['Survived']

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb 

In [18]:
# Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)

# Modelagem
Serão testados três modelos: SVM, XGBoost e RandomForest.
Começarei estimando o modelo XGBoost. Utilizarei técnicas de cross-validation para uma melhor acurácia e para melhorar os hiperparâmetros do modelo.


## Treinando XGBoost

In [19]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic')

In [20]:
xgb_model.fit(X_train, y_train)

[01:17:34] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
C:\Users\art19\anaconda3\envs\TitanicProject\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
accuracy_score(y_test, xgb_model.predict(X_test))

0.8324022346368715

## Treinando SVM
Utilizarei o GridSearchCV por se tratar de poucos hiperparâmetros para serem calibrados.

In [36]:
# Testando SVM
grid_params_svm = {
    'C':[0.001, 0.01, 0.1, 1, 10],
    'kernel':['rbf', 'linear']
    }
svm = SVC()
cv_svm_model = GridSearchCV(svm, grid_params_svm, cv=10, n_jobs=-1)

In [37]:
cv_svm_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'kernel': ['rbf', 'linear']})

In [38]:
cv_svm_model.best_estimator_

SVC(C=0.1, kernel='linear')

In [39]:
cv_svm_model.best_score_

0.8215571205007824

## Treinando RandomForest
Utilizarei técnicas de CV para calibrar os hiperparâmetros. Isso faz com que o modelo encontre uma solução melhor do que com os hiperparâmetros padrões.

In [45]:
rf = RandomForestClassifier()
param_grid_rf ={
'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
cv_rf_model = RandomizedSearchCV(rf, param_grid_rf, cv=10, n_iter=50, n_jobs=-1)

In [46]:
cv_rf_model.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [47]:
cv_rf_model.best_estimator_

RandomForestClassifier(max_depth=50, min_samples_leaf=4, n_estimators=200)

In [48]:
cv_rf_model.best_score_

0.8384389671361502

## Predizendo base de testes

Aqui aplico o melhor modelo na base de testes, para fazer a submissão no Kaggle.

In [22]:
base_para_teste = base_teste.drop(columns=['Survived', 'Age', 'Fare'], axis=1)

In [49]:
y_pred_dados_teste = cv_rf_model.predict(base_para_teste)

In [50]:
base_teste['Survived'] = y_pred_dados_teste.astype(int)

<ipython-input-50-db20a25be608>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_teste['Survived'] = y_pred_dados_teste.astype(int)


In [51]:
df_respostas_rf = pd.DataFrame()
df_respostas_rf['PassengerId'] = base_teste['PassengerId']
df_respostas_rf['Survived'] = base_teste['Survived']

In [53]:
df_respostas_xgb.to_csv('respostas_rf_titanic6.csv', index=False)

## Conclusão
Por incrível que pareça, ao submeter ao Kaggle, os três modelos tiveram exatamente a mesma perfomance: 0.78468 de acurácia. Uma acurácia considerada boa para os padrões desta competição, visto que está entre os 17% melhores da competição. Este trabalho foi muito enriquecedor e permitiu desenvolver-me nas técnicas de Machine Learning com um trabalho prático.